In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, struct, col
from pyspark.sql.types import StringType
from typing import List

spark = SparkSession.builder.appName("SalesReportXML").getOrCreate()

23/12/25 22:43:59 WARN Utils: Your hostname, kenans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.101 instead (on interface en0)
23/12/25 22:43:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/25 22:44:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [49]:
# Sample Data
data: List[Row] = [
    Row(
        ProductID="001", ProductName="Widget A", QuantitySold=10, SaleDate="2023-11-01"
    ),
    Row(ProductID="002", ProductName="Widget B", QuantitySold=5, SaleDate="2023-11-02"),
    Row(
        ProductID="003",
        ProductName="Gadget Pro",
        QuantitySold=12,
        SaleDate="2023-11-02",
    ),
]

In [50]:
def row_to_xml(row: Row) -> str:
    return f"<Sale><ProductID>{row['ProductID']}</ProductID><ProductName>{row['ProductName']}</ProductName><QuantitySold>{row['QuantitySold']}</QuantitySold><SaleDate>{row['SaleDate']}</SaleDate></Sale>"


row_to_xml_udf = udf(row_to_xml, StringType())

df = spark.createDataFrame(data)

df.show()

xml_df = df.withColumn("xml", row_to_xml_udf(struct(*df.columns)))

xml_df.select('xml').show(truncate=False)

+---------+-----------+------------+----------+
|ProductID|ProductName|QuantitySold|  SaleDate|
+---------+-----------+------------+----------+
|      001|   Widget A|          10|2023-11-01|
|      002|   Widget B|           5|2023-11-02|
|      003| Gadget Pro|          12|2023-11-02|
+---------+-----------+------------+----------+

+------------------------------------------------------------------------------------------------------------------------------------------+
|xml                                                                                                                                       |
+------------------------------------------------------------------------------------------------------------------------------------------+
|<Sale><ProductID>001</ProductID><ProductName>Widget A</ProductName><QuantitySold>10</QuantitySold><SaleDate>2023-11-01</SaleDate></Sale>  |
|<Sale><ProductID>002</ProductID><ProductName>Widget B</ProductName><QuantitySold>5</QuantitySold><

In [58]:
# Sample Data
data = [("Alice", 25), ("Bob", 29)]
df = spark.createDataFrame(data, ["name", "age"])

# Create a struct column
df_with_struct = df.withColumn("person", struct(df["name"], df["age"]))

print(df_with_struct.columns)

# df_with_struct.select("person.name", "person.age").show()



SyntaxError: cannot use starred expression here (3453714276.py, line 8)